In [1]:
import os
os.chdir(os.getcwd())

import pandas as pd # you need to also have xlrd installed
import numpy as np
from scipy.signal import argrelextrema
import xlsxwriter
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


# mean absolute deviation function
def mad(data, axis=None):
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)

speeds = [50, 100, 150, 200, 300, 400, 600, 800, 1000, 1200, 1400, 1600, 1800]
areas = [900, 1700, 2500]

surcon = 'ss perpendicular perpendicular'

In [2]:
writer = pd.ExcelWriter(surcon + '\\analysis_20190207_static.xlsx', engine = 'xlsxwriter', options={'strings_to_numbers': True})
for a in areas:
    for s in speeds:
        file = surcon + '\\' + str(a) + 'mm\\' + str(a) + 'mm_' + str(s) + 'mm_min.xls'
        xls = pd.ExcelFile(file)
        weights = [int(w[:-1]) for w in xls.sheet_names[3:]]
#         print(weights)
        if "df1" not in locals():
            df1 = pd.DataFrame(columns = speeds, index = weights)
        for k, w in enumerate(weights):
            df2 = pd.read_excel(file, sheet_name = str(w) + "g")
            
            #clean up columns
            df2.columns = ['time', 'strain', 'force', 'work']
            df2.drop(df2.index[[0, 1]], inplace = True)
            
            # static_strain(mm)','static_force(N)'
            max = df2['force'].max()
            df1[s][weights[k]] = max       

    df1.to_excel(writer, sheet_name=str(a))
    del df1
writer.save()

In [5]:
def func_lin1(x, c):
    return c * x

def func_lin2(x, a, b):
    return a * x + b

def func_TB(x, d, e):
    return d * x + e * x ** (2 / 3)

In [7]:
writer = pd.ExcelWriter(surcon + '\\analysis_20190207_kinematic_fit.xlsx', engine = 'xlsxwriter')

file = os.getcwd() + "\\" + surcon + "\\" + "analysis_20190207_kinematic.xlsx"
for a in areas:
    df6 = pd.read_excel(file, sheet_name = str(a))
    df7 = pd.DataFrame(columns = ['a', 'b', 'c', 'd', 'e'], index = speeds)
    for s in speeds:
        Force = df6[s].values.astype(float)
        popt1, pcov1 = curve_fit(func_lin1, df6.index.values, Force)
        popt2, pcov2 = curve_fit(func_lin2, df6.index.values, Force, p0 = [popt1[0], 0], bounds = ([0, 0], [np.inf, np.inf]))
        popt3, pcov3 = curve_fit(func_TB,   df6.index.values, Force, p0 = [popt1[0], 0], bounds = ([0, 0], [np.inf, np.inf]))
#         print(popt.tolist() + popt2.tolist())
        df7.loc[s] = popt1.tolist() + popt2.tolist() + popt3.tolist()
    df7.to_excel(writer, sheet_name=str(a))
writer.save()

In [3]:
writer = pd.ExcelWriter(surcon + '\\analysis_20190207_kinematic.xlsx', engine = 'xlsxwriter', options={'strings_to_numbers': True})
for a in areas:
    for s in speeds:
        file = surcon + '\\' + str(a) + 'mm\\' + str(a) + 'mm_' + str(s) + 'mm_min.xls'
        xls = pd.ExcelFile(file)
        weights = [int(w[:-1]) for w in xls.sheet_names[3:]]
#         print(weights)
        if "df1" not in locals():
            df1 = pd.DataFrame(columns = speeds, index = weights)
        for k, w in enumerate(weights):
            df2 = pd.read_excel(file, sheet_name = str(w) + "g")
            
            #clean up columns
            df2.columns = ['time', 'strain', 'force', 'work']
            df2.drop(df2.index[[0, 1]], inplace = True)
            
            n = 9  # the bigger the more comparison width
            df2['max'] = df2.iloc[argrelextrema(df2.force.values, np.greater_equal, order=n)[0], 2]
            
            #removing the NaNs
            df3 = df2.dropna(subset=['max'])
            
            #calculating the delta
            pd.options.mode.chained_assignment = None  # default='warn'
            df3['del'] = df3.strain - df3.strain.shift(1)
            
            #removing the too close points
            df3 = df3[df3['del'] > 0.5 * df3['del'].mean()]
            df3 = df3[df3['del'] < 1.9 * df3['del'].mean()]
            
            #removing NaNs
            df3 = df3.dropna(subset=['del'])
            
            df1[s][weights[k]] = df3['max'].mean()    

    df1.to_excel(writer, sheet_name=str(a))
    del df1
writer.save()